In [ ]:
from sklearn.metrics import f1_score

def calculate_f1_macro(model, X_val, y_val):
    y_pred = model.predict(X_val).argmax(axis=1)
    y_true = y_val.argmax(axis=1)
    return f1_score(y_true, y_pred, average='macro')
f1_A = calculate_f1_macro(model_A, X_A_val, y_A_val)
f1_B = calculate_f1_macro(model_B, X_B_val, y_B_val)
f1_C = calculate_f1_macro(model_C, X_C_val, y_C_val)

print("F1 Macro Modelo A:", f1_A)
print("F1 Macro Modelo B:", f1_B)
print("F1 Macro Modelo C:", f1_C)
from sklearn.metrics import classification_report

print("\n📌 Classification Report — Modelo A")
print(classification_report(y_A_val.argmax(axis=1),
                            model_A.predict(X_A_val).argmax(axis=1)))

print("\n📌 Classification Report — Modelo B")
print(classification_report(y_B_val.argmax(axis=1),
                            model_B.predict(X_B_val).argmax(axis=1)))

print("\n📌 Classification Report — Modelo C")
print(classification_report(y_C_val.argmax(axis=1),
                            model_C.predict(X_C_val).argmax(axis=1)))
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion(model, X_val, y_val, title):
    y_pred = model.predict(X_val).argmax(axis=1)
    y_true = y_val.argmax(axis=1)
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.show()

plot_confusion(model_A, X_A_val, y_A_val, "Matriz de Confusión - Modelo A")
plot_confusion(model_B, X_B_val, y_B_val, "Matriz de Confusión - Modelo B")
plot_confusion(model_C, X_C_val, y_C_val, "Matriz de Confusión - Modelo C")
# Crear tabla comparativa final
df_eval = pd.DataFrame({
    "Modelo": ["A_class_weight", "B_oversampling", "C_over+weight"],
    "Accuracy": [
        results["model_A"]["accuracy"],
        results["model_B"]["accuracy"],
        results["model_C"]["accuracy"]
    ],
    "Loss": [
        results["model_A"]["loss"],
        results["model_B"]["loss"],
        results["model_C"]["loss"]
    ],
    "Tiempo (s)": [
        results["model_A"]["time"],
        results["model_B"]["time"],
        results["model_C"]["time"]
    ],
    "F1 Macro": [f1_A, f1_B, f1_C]
})

print("\n📊 TABLA COMPARATIVA FINAL:")
display(df_eval)

# Seleccionar el mejor modelo basado en F1 Macro
best_index = df_eval["F1 Macro"].idxmax()
best_model_name = df_eval.loc[best_index, "Modelo"]
best_f1 = df_eval.loc[best_index, "F1 Macro"]

print(f"\n🏆 Mejor modelo según F1 Macro: {best_model_name}")
print(f"🔍 F1 Macro obtenido: {best_f1:.4f}")

# Asignación del modelo seleccionado
if best_model_name == "A_class_weight":
    best_model = model_A
    X_val_best = X_A_val
    y_val_best = y_A_val
elif best_model_name == "B_oversampling":
    best_model = model_B
    X_val_best = X_B_val
    y_val_best = y_B_val
else:
    best_model = model_C
    X_val_best = X_C_val
    y_val_best = y_C_val

print("\n📌 Modelo final listo para inferencias como 'best_model'")

# Diccionario de emociones
emotion_dict = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Función para limpiar texto (igual que en el dataset)
def clean_text_input(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = re.sub(r"[^A-Za-zÁÉÍÓÚÜÑáéíóúüñ\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Preparar texto para el modelo
def prepare_input(sentence):
    cleaned = clean_text_input(sentence)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad_seq = pad_sequences(seq, maxlen=max_len, padding='post')
    return pad_seq

# Función principal para predecir emoción
def predict_emotion(sentence, model):
    x_input = prepare_input(sentence)
    pred = model.predict(x_input)[0]

    emotion_id = pred.argmax()
    emotion_label = emotion_dict[emotion_id]

    print(f"\n📝 Texto ingresado:\n{sentence}")
    print(f"\n🎯 Emoción predicha: **{emotion_label.upper()}**\n")

    print("📊 Probabilidades por emoción:")
    for idx, prob in enumerate(pred):
        print(f"  {emotion_dict[idx]:10s}: {prob:.4f}")

    return emotion_label, pred

predict_emotion("I feel extremely happy today!", best_model)
predict_emotion("I'm very scared about the future...", best_model)
predict_emotion("I love the way you talk to me.", best_model)
predict_emotion("This makes me so angry!", best_model)
predict_emotion("I can't believe this happened!", best_model)

import numpy as np
import matplotlib.pyplot as plt

# Función para obtener la atención del modelo
def get_attention_weights(sentence, model):
    x_input = prepare_input(sentence)

    # Ejecutar modelo y extraer atención
    # Atención está en la capa 'attention' del modelo
    attention_layer = model.get_layer('attention')
    attention_model = tf.keras.Model(inputs=model.input, outputs=attention_layer.output)

    attention_scores = attention_model.predict(x_input)[0]  # vector tamaño max_len
    return attention_scores

# Visualizar atención como mapa de calor
def plot_attention(sentence, model):
    cleaned = clean_text_input(sentence)
    words = cleaned.split()

    attn = get_attention_weights(sentence, model)
    attn = attn[:len(words)]  # cortar al número de palabras reales

    plt.figure(figsize=(12, 1.5))
    plt.imshow([attn], cmap='viridis', aspect='auto')
    plt.colorbar()
    plt.xticks(ticks=np.arange(len(words)), labels=words, rotation=45)
    plt.yticks([])
    plt.title("Mapa de Atención — Importancia por palabra")
    plt.show()

sentence = "I feel very sad and lonely today."
plot_attention(sentence, best_model)
predict_emotion(sentence, best_model)


In [ ]:
f1_A = calculate_f1_macro(model_A, X_A_val, y_A_val)
f1_B = calculate_f1_macro(model_B, X_B_val, y_B_val)
f1_C = calculate_f1_macro(model_C, X_C_val, y_C_val)

print("F1 Macro Modelo A:", f1_A)
print("F1 Macro Modelo B:", f1_B)
print("F1 Macro Modelo C:", f1_C)


In [ ]:
from sklearn.metrics import classification_report

print("\n📌 Classification Report — Modelo A")
print(classification_report(y_A_val.argmax(axis=1),
                            model_A.predict(X_A_val).argmax(axis=1)))

print("\n📌 Classification Report — Modelo B")
print(classification_report(y_B_val.argmax(axis=1),
                            model_B.predict(X_B_val).argmax(axis=1)))

print("\n📌 Classification Report — Modelo C")
print(classification_report(y_C_val.argmax(axis=1),
                            model_C.predict(X_C_val).argmax(axis=1)))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion(model, X_val, y_val, title):
    y_pred = model.predict(X_val).argmax(axis=1)
    y_true = y_val.argmax(axis=1)
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.show()

plot_confusion(model_A, X_A_val, y_A_val, "Matriz de Confusión - Modelo A")
plot_confusion(model_B, X_B_val, y_B_val, "Matriz de Confusión - Modelo B")
plot_confusion(model_C, X_C_val, y_C_val, "Matriz de Confusión - Modelo C")


In [ ]:
# Crear tabla comparativa final
df_eval = pd.DataFrame({
    "Modelo": ["A_class_weight", "B_oversampling", "C_over+weight"],
    "Accuracy": [
        results["model_A"]["accuracy"],
        results["model_B"]["accuracy"],
        results["model_C"]["accuracy"]
    ],
    "Loss": [
        results["model_A"]["loss"],
        results["model_B"]["loss"],
        results["model_C"]["loss"]
    ],
    "Tiempo (s)": [
        results["model_A"]["time"],
        results["model_B"]["time"],
        results["model_C"]["time"]
    ],
    "F1 Macro": [f1_A, f1_B, f1_C]
})

print("\n📊 TABLA COMPARATIVA FINAL:")
display(df_eval)


In [ ]:
# Seleccionar el mejor modelo basado en F1 Macro
best_index = df_eval["F1 Macro"].idxmax()
best_model_name = df_eval.loc[best_index, "Modelo"]
best_f1 = df_eval.loc[best_index, "F1 Macro"]

print(f"\n🏆 Mejor modelo según F1 Macro: {best_model_name}")
print(f"🔍 F1 Macro obtenido: {best_f1:.4f}")

# Asignación del modelo seleccionado
if best_model_name == "A_class_weight":
    best_model = model_A
    X_val_best = X_A_val
    y_val_best = y_A_val
elif best_model_name == "B_oversampling":
    best_model = model_B
    X_val_best = X_B_val
    y_val_best = y_B_val
else:
    best_model = model_C
    X_val_best = X_C_val
    y_val_best = y_C_val

print("\n📌 Modelo final listo para inferencias como 'best_model'")


In [ ]:
# Diccionario de emociones
emotion_dict = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Función para limpiar texto (igual que en el dataset)
def clean_text_input(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = re.sub(r"[^A-Za-zÁÉÍÓÚÜÑáéíóúüñ\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Preparar texto para el modelo
def prepare_input(sentence):
    cleaned = clean_text_input(sentence)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad_seq = pad_sequences(seq, maxlen=max_len, padding='post')
    return pad_seq

# Función principal para predecir emoción
def predict_emotion(sentence, model):
    x_input = prepare_input(sentence)
    pred = model.predict(x_input)[0]

    emotion_id = pred.argmax()
    emotion_label = emotion_dict[emotion_id]

    print(f"\n📝 Texto ingresado:\n{sentence}")
    print(f"\n🎯 Emoción predicha: **{emotion_label.upper()}**\n")

    print("📊 Probabilidades por emoción:")
    for idx, prob in enumerate(pred):
        print(f"  {emotion_dict[idx]:10s}: {prob:.4f}")

    return emotion_label, pred


In [ ]:
predict_emotion("I feel extremely happy today!", best_model)
predict_emotion("I'm very scared about the future...", best_model)
predict_emotion("I love the way you talk to me.", best_model)
predict_emotion("This makes me so angry!", best_model)
predict_emotion("I can't believe this happened!", best_model)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Función para obtener la atención del modelo
def get_attention_weights(sentence, model):
    x_input = prepare_input(sentence)

    # Ejecutar modelo y extraer atención
    # Atención está en la capa 'attention' del modelo
    attention_layer = model.get_layer('attention')
    attention_model = tf.keras.Model(inputs=model.input, outputs=attention_layer.output)

    attention_scores = attention_model.predict(x_input)[0]  # vector tamaño max_len
    return attention_scores

# Visualizar atención como mapa de calor
def plot_attention(sentence, model):
    cleaned = clean_text_input(sentence)
    words = cleaned.split()

    attn = get_attention_weights(sentence, model)
    attn = attn[:len(words)]  # cortar al número de palabras reales

    plt.figure(figsize=(12, 1.5))
    plt.imshow([attn], cmap='viridis', aspect='auto')
    plt.colorbar()
    plt.xticks(ticks=np.arange(len(words)), labels=words, rotation=45)
    plt.yticks([])
    plt.title("Mapa de Atención — Importancia por palabra")
    plt.show()


In [ ]:
sentence = "I feel very sad and lonely today."
plot_attention(sentence, best_model)
predict_emotion(sentence, best_model)
